In [30]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import metrics
import utils

# Decorrelation

Let $X \in \mathbb{R}^{p*N}$ a matrix of $N$ observations with $p$ features. We suppose the matrix $X$ is centered.  
The goal of decorrelation is to transform $X$ into a new matrix with a diagonal covariance matrix.  
We need to find $W_D \in \mathbb{R}^{p*p}$ such that:
$$Y = W_D X$$
And the components of $Y$ are uncorrelated, that is:
$$D = \text{cov}(X)$$  

Let's define the covariance matrix of $X$ is:
$$\Sigma = \text{cov}(X) = E[XX^T] \approx \frac{1}{N} XX^T$$
$\Sigma$ is positive semi-definite, we can diagonalize it to get:
$$\Sigma = E^TDE$$
$$E^T\Sigma E=D$$  

Solving for $D = \text{cov}(Y)$, we get:
$$W_D = E^T$$

In [34]:
def decorr(X):
    C = X @ X.T
    d, E = np.linalg.eigh(C)
    return E.T

X = np.random.randn(4, 37)
X -= np.mean(X, axis=1, keepdims=True)
W = decorr(X)
Y = W @ X
print(Y.shape)
print(np.cov(Y, bias=True))

(4, 37)
[[ 7.12485051e-01  2.71748829e-16  2.24455308e-16  1.29001442e-18]
 [ 2.71748829e-16  9.84199824e-01 -9.85786674e-17 -4.81641620e-16]
 [ 2.24455308e-16 -9.85786674e-17  1.08493289e+00  1.80692545e-16]
 [ 1.29001442e-18 -4.81641620e-16  1.80692545e-16  1.51172982e+00]]


# Whitening

The goal of decorrelation is to transform $X$ into a new matrix with a unit diagonal covariance matrix.  
We need to find $W \in \mathbb{R}^{p*p}$ such that:
$$Y = W X$$
And the components of $Y$ are uncorrelated with unit variance, that is:
$$\text{cov}(X) = I_p$$

Using the decorrelation method above, we get $W_D$ and $Y$ has uncorrelated components. we just need to divide each component by it's standard deviation in order to get unit variance:
$$W = D^{-1/2}E^T$$

In [54]:
def whitening(X):
    N = X.shape[1]
    C = (X @ X.T) / N
    d, E = np.linalg.eigh(C)
    return E.T / np.sqrt(d).reshape(-1,1)
    

X = np.random.randn(4, 37)
X -= np.mean(X, axis=1, keepdims=True)
W = whitening(X)
Y = W @ X
print(Y.shape)
C = np.cov(Y, bias=True)
print(C)
print(metrics.tdist(C, np.eye(len(C))))

(4, 37)
[[ 1.00000000e+00 -1.15878238e-17 -2.46883269e-16  1.15772493e-16]
 [-1.15878238e-17  1.00000000e+00  1.30050339e-16  2.74140490e-16]
 [-2.46883269e-16  1.30050339e-16  1.00000000e+00 -2.42384455e-16]
 [ 1.15772493e-16  2.74140490e-16 -2.42384455e-16  1.00000000e+00]]
1.4499268802213633e-15
